In [39]:
import pandas as pd

In [40]:
# switch the source depending on the hospital your looking for
bi_neg_cohort = pd.read_csv('/home/jsearle/bigDrive/NAX/NLP-SAH_identification/cleanCohorts/BIDMC_neg_cohort_final_updated.csv')
bi_pos_cohort = pd.read_csv('/home/jsearle/bigDrive/NAX/NLP-SAH_identification/cleanCohorts/BIDMC_pos_cohort_final_updated.csv')

mgb_neg_cohort = pd.read_csv('/home/jsearle/bigDrive/NAX/NLP-SAH_identification/cleanCohorts/MGB_neg_cohort_final.csv')
mgb_pos_cohort = pd.read_csv('/home/jsearle/bigDrive/NAX/NLP-SAH_identification/cleanCohorts/MGB_pos_cohort_final.csv')

# Select the BDSPPatientID column from each cohort
bi_neg_ids = bi_neg_cohort[['BDSPPatientID']]
bi_pos_ids = bi_pos_cohort[['BDSPPatientID']]

mgb_neg_ids = mgb_neg_cohort[['BDSPPatientID']]
mgb_pos_ids = mgb_pos_cohort[['BDSPPatientID']]

# Combine the neg cohorts
combined_neg_cohort = pd.concat([bi_neg_ids, mgb_neg_ids], ignore_index=True)

# Combine the pos cohorts
combined_pos_cohort = pd.concat([bi_pos_ids, mgb_pos_ids], ignore_index=True)

fn = pd.read_csv('/home/jsearle/bigDrive/NAX/NLP-SAH_identification/finalModel/results/LogReg/false_negatives.csv')
fp = pd.read_csv('/home/jsearle/bigDrive/NAX/NLP-SAH_identification/finalModel/results/LogReg/false_positives.csv')
print(len(fn))
print(len(fp))

23
100


In [41]:
all_errors = pd.concat([fn, fp], ignore_index=True)
print(len(all_errors))
all_errors.head()

123


,BDSPPatientID
0,115345969
1,150016455
2,120391421
3,118263835
4,117223743


In [42]:
# Function to calculate error rates
def calculate_error_rates(neg_cohort, pos_cohort, all_errors):
    # Calculate the total number of patients in each cohort
    total_neg_patients = len(neg_cohort)
    total_pos_patients = len(pos_cohort)

    # Calculate the number of errors in each cohort
    neg_errors = len(neg_cohort[neg_cohort['BDSPPatientID'].isin(all_errors['BDSPPatientID'])])
    pos_errors = len(pos_cohort[pos_cohort['BDSPPatientID'].isin(all_errors['BDSPPatientID'])])

    # Calculate the error rates
    neg_error_rate = neg_errors / total_neg_patients if total_neg_patients > 0 else 0
    pos_error_rate = pos_errors / total_pos_patients if total_pos_patients > 0 else 0

    # Display the results
    print(f'Negative Cohort Error Rate: {neg_errors}/{total_neg_patients} = {neg_error_rate}')
    print(f'Positive Cohort Error Rate: {pos_errors}/{total_pos_patients} = {pos_error_rate}')

    return neg_errors, total_neg_patients, pos_errors, total_pos_patients

In [43]:
# Calculate error rates for each hospital
print('BIDMC')
bi_neg_errors, bi_total_neg, bi_pos_errors, bi_total_pos = calculate_error_rates(bi_neg_cohort, bi_pos_cohort, all_errors)

print('\nMGB')
mgb_neg_errors, mgb_total_neg, mgb_pos_errors, mgb_total_pos = calculate_error_rates(mgb_neg_cohort, mgb_pos_cohort, all_errors)

print('\nCombined')
combined_neg_errors, combined_total_neg, combined_pos_errors, combined_total_pos = calculate_error_rates(combined_neg_cohort, combined_pos_cohort, all_errors)



BIDMC
Negative Cohort Error Rate: 0/499 = 0.0
Positive Cohort Error Rate: 30/515 = 0.05825242718446602

MGB
Negative Cohort Error Rate: 1/1029 = 0.0009718172983479105
Positive Cohort Error Rate: 92/1053 = 0.08736942070275404

Combined
Negative Cohort Error Rate: 1/1528 = 0.0006544502617801048
Positive Cohort Error Rate: 122/1568 = 0.0778061224489796


In [47]:
#Calculate Overall Error Rate
# P[E] = (Pe+ * p+) + (Pe- * p-)

# Calculate the overall error rate for MGB
mgb_pos_prevalence = 85 / 10000
mgb_neg_prevalence = 1 - mgb_pos_prevalence
mgb_error_rate = (mgb_pos_errors / mgb_total_pos * mgb_pos_prevalence) + (mgb_neg_errors / mgb_total_neg * mgb_neg_prevalence)
print(f'MGB Error Rate: {mgb_error_rate * 100:.2f}%')


MGB Error Rate: 0.17%


In [45]:
# Calculate the overall error rate for BIDMC
bi_pos_prevalence = 21 / 10000
bi_neg_prevalence = 1 - bi_pos_prevalence
bi_error_rate = (bi_pos_errors / bi_total_pos * bi_pos_prevalence) + (bi_neg_errors / bi_total_neg * bi_neg_prevalence)
print(f'BIDMC Error Rate: {bi_error_rate * 100:.2f}%')

BIDMC Error Rate: 0.01%


In [46]:
# Calculate the overall error rate for the combined cohorts
combined_pos_prevalence = (85 + 21) / (10000 + 10000)
combined_neg_prevalence = 1 - combined_pos_prevalence
combined_error_rate = (combined_pos_errors / combined_total_pos * combined_pos_prevalence) + (combined_neg_errors / combined_total_neg * combined_neg_prevalence)
print(f'Combined Error Rate: {combined_error_rate * 100:.2f}%')

Combined Error Rate: 0.11%
